In [17]:
!nvidia-smi
import torch
torch.cuda.is_available()

Thu Mar 20 07:37:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P0             26W /   70W |    3369MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

True

In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olivetti-faces/olivetti_faces_target.npy
/kaggle/input/olivetti-faces/olivetti_faces.npy


In [19]:
import torch # 引入PyTorch
import torch.nn as nn   # 引入神经网络模块
import torch.optim as optim # 引入优化器
from torch.utils.data import DataLoader  # 数据加载器
from sklearn.datasets import fetch_olivetti_faces # 引入Olivetti人脸数据集
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
olivetti_faces = fetch_olivetti_faces(data_home='./', shuffle=True)
# print(olivetti_faces.data.shape)
LR = 1e-3
epochs = 10
BATCH_SIZE = 20
olivetti_faces = fetch_olivetti_faces(data_home='./', shuffle=True)
X = olivetti_faces.data
y = olivetti_faces.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train =torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
train_dataset = [(img,lbl) for img,lbl in zip(x_train, y_train)]
test_dataset = [(img,lb1) for img,lb1 in zip(x_test,y_test)]
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader =  torch.utils.data.DataLoader(test_dataset,batch_size=BATCH_SIZE,shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [20]:

len(test_dataset)

80

In [21]:
modelA =nn.Sequential(
    nn.Linear(4096,12288),
    nn.BatchNorm1d(12288),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(12288,8192),
    nn.BatchNorm1d(8192),
    nn.ReLU(),
    nn.Dropout(0.6),
    nn.Linear(8192,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(2048,40)
).to(device)

In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelA.parameters(), lr=LR)


In [23]:
loss_history = []
for epoch in range(epochs):
    modelA.train()
    for data,target in train_loader:
        data,target = data.to(device),target.to(device)
        output = modelA(data)
        loss = loss_fn(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
        print(f'Epoch:{epoch} Loss: {loss.item()}')


Epoch:0 Loss: 3.9406819343566895
Epoch:0 Loss: 3.360774278640747
Epoch:0 Loss: 3.3279178142547607
Epoch:0 Loss: 3.5889599323272705
Epoch:0 Loss: 3.606302261352539
Epoch:0 Loss: 3.6377854347229004
Epoch:0 Loss: 2.9178056716918945
Epoch:0 Loss: 3.4478373527526855
Epoch:0 Loss: 2.8338420391082764
Epoch:0 Loss: 3.506864547729492
Epoch:0 Loss: 2.402233600616455
Epoch:0 Loss: 3.2417144775390625
Epoch:0 Loss: 3.6580562591552734
Epoch:0 Loss: 2.33790922164917
Epoch:0 Loss: 2.176440715789795
Epoch:0 Loss: 1.8577759265899658
Epoch:1 Loss: 2.6433091163635254
Epoch:1 Loss: 1.987426519393921
Epoch:1 Loss: 1.8309288024902344
Epoch:1 Loss: 2.5664868354797363
Epoch:1 Loss: 1.6831413507461548
Epoch:1 Loss: 1.8029029369354248
Epoch:1 Loss: 2.2905466556549072
Epoch:1 Loss: 1.456850528717041
Epoch:1 Loss: 2.1085057258605957
Epoch:1 Loss: 2.037972927093506
Epoch:1 Loss: 2.61265230178833
Epoch:1 Loss: 2.1544487476348877
Epoch:1 Loss: 1.8158451318740845
Epoch:1 Loss: 1.7742030620574951
Epoch:1 Loss: 1.682228

In [24]:
correct = 0
total = 0
modelA.eval() # train(False)
with torch.no_grad():  
    for img2, target2 in test_loader:
        img2,target2 = img2.to(device),target2.to(device)
        output = modelA(img2.reshape(img2.size(0), -1)) 
        _, predicted = torch.max(output, 1)  
        total += target2.size(0)  
        correct += (predicted == target2).sum().item()

print(f'Accuracy: {correct/total*100}%')

Accuracy: 77.5%


In [27]:
# 保存模型
torch.save(modelA.state_dict(), '/kaggle/working/olivetti_faces_model.pth')